In [8]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
import os
import matplotlib.colors as mcolors

from ipynb.fs.full.functions_for_creating_plots import *
os.chdir(os.getcwd())

In [72]:
## Dictionary holding the numbers of the molecules in the model (for explanation, look up model-figure.pdf)
molecules_dict={'$\mathbf{G_{GTP}}$':[23,33],
                '$\mathbf{IP_{3}}$':[40],
                'Total Arrestin-Receptor':[10,11,12,13,14,15],
                'PM Arrestin-Receptor':[10,11,12],
                'IC Arrestin-Receptor':[13,14,15]}

                
perturbed_processes={'Koff_Kon_all':{'Koff_Kon_perturbations':{ 'kon_values':[0.03,0.3,3],
                                                                'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                'koff_values':[0.0003,0.03,0.3],
                                                                'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                     'Process_model_numbers':[]},

                    'Phosphorylation_PM':{'Koff_Kon_perturbations':{'kon_values':[0.3],
                                                                    'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                    'koff_values':[0.0003,0.03,0.3],
                                                                    'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                        'Process_model_numbers':[30]},

                    'Phosphorylation_IC':{'Koff_Kon_perturbations':{'kon_values':[0.3],
                                                                    'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                    'koff_values':[0.0003,0.03,0.3],
                                                                    'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                        'Process_model_numbers':[50]},

                    'Phosphorylation_IC_PM':{'Koff_Kon_perturbations':{ 'kon_values':[0.3],
                                                                        'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                        'koff_values':[0.0003,0.03,0.3],
                                                                        'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                            'Process_model_numbers':[50,30]},

                    'Phosphatase_PM':{'Koff_Kon_perturbations':{'kon_values':[0.3],
                                                                'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                'koff_values':[0.0003,0.03,0.3],
                                                                'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                      'Process_model_numbers':[25,27,29]},

                    'Phosphatase_IC':{'Koff_Kon_perturbations':{'kon_values':[0.3],
                                                                'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                'koff_values':[0.0003,0.03,0.3],
                                                                'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                      'Process_model_numbers': [47,48,49]},  

                    'Phosphatase_IC_PM':{'Koff_Kon_perturbations':{ 'kon_values':[0.3],
                                                                    'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                    'koff_values':[0.0003,0.03,0.3],
                                                                    'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                        'Process_model_numbers': [25,27,29,47,48,49]},

                    'Arrestin_association':{'Koff_Kon_perturbations':{  'kon_values':[0.3],
                                                                        'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                        'koff_values':[0.0003,0.03,0.3],
                                                                        'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                            'Process_model_numbers': [35,44,36,37,45,46]},

                    'Arrestin_dissociation':{'Koff_Kon_perturbations':{ 'kon_values':[0.3],
                                                                        'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                        'koff_values':[0.0003,0.03,0.3],
                                                                        'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                             'Process_model_numbers': [32,33,41,42,34,43]},

                    'Internalisation':{'Koff_Kon_perturbations':{   'kon_values':[0.3],
                                                                    'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                    'koff_values':[0.0003,0.03,0.3],
                                                                    'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                        'Process_model_numbers': [38, 39, 40]}}

## Determine ligand concentrations              
ligand_ccs=(10 ** np.arange(-6, 1.1, 1))
internalisation_status=['With_internalisation','No_internalisation']

## Simulation time
simulation_time=1200

In [102]:
def draw_kinetic_concentration_curve_of_molecule_with_different_ligand_ccs(molecule,molecules_dict,kon,koff,intern_status,process_name,ligand_ccs,\
                                                                           color_palette_dict,color_names,ax,subplot_name,fig_width,fig_height,\
                                                                           subplot_title_fontsize_nominator,subplot_axis_label_fontsize_nominator,\
                                                                           subplot_axis_tick_label_fontsize_nominator,subplot_legend_label_fontsize_nominator,\
                                                                           legend_title,subplot_legend_title_fontsize_nominator):
    ## Create directory path to load the data from
    dirname=os.path.join('../data/',intern_status,process_name,'')

    ## Adjust ligand concentrations to used Kon and Koff values +  convert it from uM/L to M/L (by subtracting 6 from hte logarithmic values of ligand ccs)
    ligands=np.log10(adjust_ligand_ccs(ligand_ccs,koff,kon)) - 6

    ## Define which subplot we eant to plot into from the original subplot_mosaic object
    axis=ax[subplot_name]

    ## Get color palette 
    colors = [color_palette_dict.get(col) for col in color_names]

    ## Loop over the ligand concentrations, load the respective simulation data from the .csv file
    for ligand,color in zip(ligands,colors):
        fname=dirname+ '_'.join([process_name,'koff',str(koff),'kon',str(kon),'ligand_cc',str(ligand)]) + '.csv'
        col_nums=[int(x+1) for x in molecules_dict[molecule]]
        col_nums.append(0)
        mol_ccs_df=pd.read_csv(fname, index_col=0,usecols=col_nums)
        axis.plot(mol_ccs_df.index.tolist(),np.sum(mol_ccs_df.loc[:,:],1),label=ligand,color=color)
    
    ## Customize legend, plot title, axis labels 
    plot_area=fig_width*fig_height

    axis.legend(title=legend_title,title_fontsize=plot_area/subplot_legend_title_fontsize_nominator,\
                fontsize=plot_area/subplot_legend_label_fontsize_nominator)
    
    axis_label_fontsize=plot_area/subplot_axis_label_fontsize_nominator
    axis_tick_labelsize=plot_area/subplot_axis_tick_label_fontsize_nominator

    title="$\mathbf{Low\ K_{off}}$"
    title_fontsize=plot_area/subplot_title_fontsize_nominator
    axis.set_title(title,fontsize=title_fontsize)

    axis.set_xlabel('Time (s)',fontweight="bold",fontsize=axis_label_fontsize)
    axis.set_ylabel(molecule,fontweight="bold",fontsize=axis_label_fontsize)
    axis.tick_params(axis='both', labelsize=axis_tick_labelsize)


### Add plots for the first row (subplots **B - D**)

In [ ]:
subplots_names=subplot_layout[0][1:]
molecules_to_plot=['$\mathbf{G_{GTP}}$','$\mathbf{IP_{3}}$','Total Arrestin-Receptor']

kon=0.3
koff=0.0003
intern_status='With_internalisation'
process_name='Koff_Kon_all'
color_palette_dict=mcolors.CSS4_COLORS
color_names=['tomato','peru','saddlebrown','darkorange','forestgreen','turquoise','royalblue','violet']


subplot_layout=[['.','B','C','D'],
                ['E_up','F_up','G_up','H_up'],
                ['E_low','F_low','G_low','H_low'],
                ['I','J','K','K'],
                ['L','L','M','M']]
fig_width=40
fig_height=40
fig,ax=plt.subplot_mosaic(subplot_layout,figsize=(fig_width,fig_height))

## FOntsize=fig_width*fig_height/title_fontsize_ratio
subplot_title_fontsize_nominator=70
subplot_axis_label_fontsize_nominator=70
subplot_axis_tick_label_fontsize_nominator=85
subplot_legend_label_fontsize_nominator=115
legend_title='$\mathbf{[Ligand]\ (M)}$'
subplot_legend_title_fontsize_nominator=100


for subplot_name,molecule in zip(subplots_names,molecules_to_plot):
    draw_kinetic_concentration_curve_of_molecule_with_different_ligand_ccs(molecule,molecules_dict,kon,koff,intern_status,process_name,ligand_ccs,\
                                                                           color_palette_dict,color_names,ax,subplot_name,fig_width,fig_height,\
                                                                           subplot_title_fontsize_nominator,subplot_axis_label_fontsize_nominator,\
                                                                           subplot_axis_tick_label_fontsize_nominator,subplot_legend_label_fontsize_nominator,\
                                                                           legend_title,subplot_legend_title_fontsize_nominator)
                                                                          
plt.show()


### Add plots for the second and third row (**E - G**)

In [ ]:
def draw_kinetic_concentration_curve_of_molecule_with_different_koff_values(molecule,molecules_dict,kon,koff_list,intern_status,process_name,ligand_ccs,\
                                                                           color_palette_dict,color_names,ax,subplot_name,fig_width,fig_height,\
                                                                           subplot_title_fontsize_nominator,subplot_axis_label_fontsize_nominator,\
                                                                           subplot_axis_tick_label_fontsize_nominator,subplot_legend_label_fontsize_nominator,\
                                                                           legend_title,subplot_legend_title_fontsize_nominator):
    ## Create directory path to load the data from
    dirname=os.path.join('../data/',intern_status,process_name,'')

    ## Adjust ligand concentrations to used Kon and Koff values +  convert it from uM/L to M/L (by subtracting 6 from hte logarithmic values of ligand ccs)
    ligands=np.log10(adjust_ligand_ccs(ligand_ccs,koff,kon)) - 6

    ## Define which subplot we eant to plot into from the original subplot_mosaic object
    axis=ax[subplot_name]

    ## Get color palette 
    colors = [color_palette_dict.get(col) for col in color_names]

    ## Loop over the ligand concentrations, load the respective simulation data from the .csv file
    for koff in koff_list:
        for ligand,color in zip(ligands,colors):
            fname=dirname+ '_'.join([process_name,'koff',str(koff),'kon',str(kon),'ligand_cc',str(ligand)]) + '.csv'
            col_nums=[int(x+1) for x in molecules_dict[molecule]]
            col_nums.append(0)
            mol_ccs_df=pd.read_csv(fname, index_col=0,usecols=col_nums)
            axis.plot(mol_ccs_df.index.tolist(),np.sum(mol_ccs_df.loc[:,:],1),label=ligand,color=color)
    
    ## Customize legend, plot title, axis labels 
    plot_area=fig_width*fig_height

    axis.legend(title=legend_title,title_fontsize=plot_area/subplot_legend_title_fontsize_nominator,\
                fontsize=plot_area/subplot_legend_label_fontsize_nominator)
    
    axis_label_fontsize=plot_area/subplot_axis_label_fontsize_nominator
    axis_tick_labelsize=plot_area/subplot_axis_tick_label_fontsize_nominator

    title="$\mathbf{Low\ K_{off}}$"
    title_fontsize=plot_area/subplot_title_fontsize_nominator
    axis.set_title(title,fontsize=title_fontsize)

    axis.set_xlabel('Time (s)',fontweight="bold",fontsize=axis_label_fontsize)
    axis.set_ylabel(molecule,fontweight="bold",fontsize=axis_label_fontsize)
    axis.tick_params(axis='both', labelsize=axis_tick_labelsize)

In [ ]:
kon=0.3
process_name='Koff_Kon_all'
dirname='../data/With_internalisation/Koff_Kon_all/'
import matplotlib.colors as mcolors

ligand_ccs=(10 ** np.arange(-6, 1.1, 1))
fig,ax=plt.subplots(1,3,figsize=(17.5,6.25))
fig2,ax2=plt.subplots(1,3,figsize=(17.5,6.25))
for n,koff in enumerate([0.0003,0.03,0.3]):
    
    Kd_ratio = (koff/ kon) / (0.0003 / 0.3)
    ligands = ligand_ccs * Kd_ratio
    ligands=(np.log10(ligands) - 6)
    colors=[*mcolors.XKCD_COLORS][25:25+len(ligands)]
    for ligand,color in zip(ligands,colors):
        fname=dirname+ '_'.join([process_name,'koff',str(koff),'kon',str(kon),'ligand_cc',str(ligand)]) + '.csv'
        col_nums=[int(x+1) for x in molecules_dict['Arrestin-Receptor']]
        col_nums.append(0)
        arr_receptor_df=pd.read_csv(fname, index_col=0,usecols=col_nums)
        ax[n].plot(arr_receptor_df.index.tolist(),np.sum(arr_receptor_df.loc[:,:],1),label=ligand,color=mcolors.XKCD_COLORS[color])
        ax2[n].scatter(ligand,np.sum(arr_receptor_df.iloc[-1,:]),label=ligand,color=mcolors.XKCD_COLORS[color])
    ax[n].legend()
    title=' '.join(['Koff',str(koff),'Kon:',str(kon)])
    ax[n].set_title(title)
    ax[n].set_xlabel('Time (s)',fontweight="bold")
    ax[n].set_ylabel('Arrestin-Receptor',fontweight="bold",fontsize=11)
    ax[n].set_ylim(0,4500)
    ax2[n].legend(ligands)
    ax2[n].set_title(title)
    ax2[n].set_xlabel('log10 Ligand [mol/l]',fontweight="bold")
    ax2[n].set_ylabel('Arrestin-Receptor at '+str(int(arr_receptor_df.index.tolist()[-1])) +' sec',fontweight="bold",fontsize=11)
    ax2[n].set_ylim(-50,4500)
fig.savefig('../data/control_figs/plot2.png',dpi=300)
fig2.savefig('../data/control_figs/plot3.png',dpi=300)